## Clasificación binaria: 1 neurona con varias columnas en X

## 1. Dataset

dataset ficticio para clasificación.

Suponemos que tenemos dos variables de entrada:
- $ x_1 $ = Colesterol,  
- $ x_2 $ = Glucosa,

y la salida será $ y \in \{0,1\} $, donde 0 significa "no apto" y 1 significa "apto".  

In [8]:
import numpy as np

# X con forma (N, 2). Cada fila es [colesterol, glucosa].
X = np.array([
    [180, 85],
    [200, 90],
    [220, 120],
    [250, 130],
    [270, 150],
    [300, 180]
], dtype=float)

# Etiquetas (0: no apto, 1: apto)
y = np.array([0, 0, 1, 1, 1, 1], dtype=float)

N = X.shape[0]  # Número de ejemplos

## Definir neurona

Matemáticamente, una **neurona** con **función sigmoide** (logistic) hace:

$$
z^{(i)} = w_1 \cdot x_1^{(i)} + w_2 \cdot x_2^{(i)} + b,
$$
$$
\hat{y}^{(i)} = \sigma\bigl(z^{(i)}\bigr) = \frac{1}{1 + e^{-\,z^{(i)}}}.
$$

- $ \mathbf{w} = (w_1, w_2) $ son los pesos,  
- $ b $ es el sesgo (bias),
- $ \hat{y}^{(i)} $ es la **probabilidad** de que el ejemplo $ i $ pertenezca a la clase 1 (apto).

In [9]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def forward(X, w, b):
    z = np.dot(X, w) + b  # (N,) = w1*X[:,0] + w2*X[:,1] + b
    return sigmoid(z)

## Función de costo: Entropía Cruzada Binaria

Para la **clasificación binaria**, la pérdida típica es la **binary cross-entropy** (BCE):
$$
\text{BCE} = - \frac{1}{N} \sum_{i=1}^N \Bigl[y^{(i)} \log(\hat{y}^{(i)}) + \bigl(1 - y^{(i)}\bigr)\log\bigl(1 - \hat{y}^{(i)}\bigr)\Bigr].
$$
Este costo penaliza las predicciones que se alejan de la verdadera etiqueta (0 o 1).


In [10]:
def binary_cross_entropy(y_pred, y_true):
    epsilon = 1e-7
    # Clampeamos la predicción para evitar log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

## Gradientes

Con la neurona logística, es conocido que:

$$
\frac{\partial \text{BCE}}{\partial w_j}
= \frac{1}{N} \sum_{i=1}^N (\hat{y}^{(i)} - y^{(i)}) \, x_j^{(i)},
$$

para $ j = 1, 2 $. En forma **vectorizada**:

$$
\nabla_{\mathbf{w}} \text{BCE} 
= \frac{1}{N} \, X^\top \bigl(\hat{y} - y\bigr).
$$

Igualmente, respecto al sesgo $ b $:

$$
\frac{\partial \text{BCE}}{\partial b}
= \frac{1}{N} \sum_{i=1}^N 
     (\hat{y}^{(i)} - y^{(i)}).
$$

Estas derivadas permiten usar **descenso por gradiente** para actualizar $ w_1, w_2 $ y $ b $.

In [11]:
def grad_w(X, y, w, b):
    """
    Derivada de la BCE wrt w (tamaño (2,)).
    """
    y_pred = forward(X, w, b)  # shape (N,)
    # (y_pred - y) -> shape (N,)
    # X.T -> shape (2, N)
    # np.dot(X.T, y_pred - y) -> shape (2,)
    return np.dot(X.T, (y_pred - y)) / len(X)

def grad_b(X, y, w, b):
    """
    Derivada de la BCE wrt b (escalar).
    """
    y_pred = forward(X, w, b)
    return np.mean(y_pred - y)  # 1/N * sum(y_pred - y)

## Entrenamiento

In [12]:
np.random.seed(42)
w = np.random.randn(2) * 0.01  # inicialización aleatoria pequeña
b = 0.0                        # sesgo en 0 para empezar (podríamos poner algo aleatorio también)

learning_rate = 0.01
num_epochs = 1000

for epoch in range(num_epochs):
    y_pred = forward(X, w, b)
    loss_value = binary_cross_entropy(y_pred, y)
    
    dw = grad_w(X, y, w, b)
    db = grad_b(X, y, w, b)
    
    w -= learning_rate * dw
    b -= learning_rate * db
    
    # (opcional) cada cierto número de épocas
    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss={loss_value:.6f}, w={w}, b={b:.4f}")

print("\nEntrenamiento terminado.")
print("Pesos finales (w):", w)
print("Sesgo final (b):", b)

Epoch 50/1000, Loss=0.002318, w=[-0.49469323  0.99119849], b=-0.0205
Epoch 100/1000, Loss=0.002265, w=[-0.49699163  0.99579701], b=-0.0207
Epoch 150/1000, Loss=0.002214, w=[-0.49923793  1.00029127], b=-0.0209
Epoch 200/1000, Loss=0.002166, w=[-0.50143443  1.00468592], b=-0.0211
Epoch 250/1000, Loss=0.002120, w=[-0.50358331  1.00898529], b=-0.0213
Epoch 300/1000, Loss=0.002075, w=[-0.50568659  1.01319344], b=-0.0214
Epoch 350/1000, Loss=0.002033, w=[-0.50774618  1.01731417], b=-0.0216
Epoch 400/1000, Loss=0.001992, w=[-0.50976386  1.02135104], b=-0.0218
Epoch 450/1000, Loss=0.001953, w=[-0.5117413   1.02530743], b=-0.0220
Epoch 500/1000, Loss=0.001915, w=[-0.51368009  1.02918647], b=-0.0221
Epoch 550/1000, Loss=0.001879, w=[-0.51558172  1.03299116], b=-0.0223
Epoch 600/1000, Loss=0.001844, w=[-0.51744757  1.03672429], b=-0.0224
Epoch 650/1000, Loss=0.001810, w=[-0.51927899  1.04038851], b=-0.0226
Epoch 700/1000, Loss=0.001778, w=[-0.52107722  1.04398633], b=-0.0227
Epoch 750/1000, Loss=

In [13]:
y_pred_final = forward(X, w, b)
print("Predicciones (prob):", y_pred_final)
print("Clases predichas   :", (y_pred_final >= 0.5).astype(int))
print("Clases reales      :", y)

Predicciones (prob): [5.59893245e-03 2.80166144e-05 9.99980040e-01 9.96041228e-01
 9.99999907e-01 1.00000000e+00]
Clases predichas   : [0 0 1 1 1 1]
Clases reales      : [0. 0. 1. 1. 1. 1.]


In [14]:
def accuracy_score(y_true, y_pred):
    # Convertir predicciones a clases binarias (0 o 1) si son probabilidades
    y_pred_classes = (y_pred >= 0.5).astype(int)
    
    # Calcular el número de aciertos
    correct_predictions = np.sum(y_true == y_pred_classes)
    
    # Calcular accuracy
    accuracy = correct_predictions / len(y_true)
    return accuracy

accuracy_value = accuracy_score(y, (y_pred_final >= 0.5).astype(int))
print(f"Accuracy: {accuracy_value:.2%}")

Accuracy: 100.00%
